# MODEL - IMAGE LOADING & NEURAL NETWORK

In [1]:
#Import libraries
import gc
import csv
import os
import io
import cv2
from PIL import Image
import h5py
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorrt

2024-09-23 20:02:50.066977: I tensorflow/core/platform/cpu_feature_guard.cc:181] Beginning TensorFlow 2.15, this package will be updated to install stock TensorFlow 2.15 alongside Intel's TensorFlow CPU extension plugin, which provides all the optimizations available in the package and more. If a compatible version of stock TensorFlow is present, only the extension will get installed. No changes to code or installation setup is needed as a result of this change.
More information on Intel's optimizations for TensorFlow, delivered as TensorFlow extension plugin can be viewed at https://github.com/intel/intel-extension-for-tensorflow.
2024-09-23 20:02:50.067028: I tensorflow/core/platform/cpu_feature_guard.cc:192] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## 1) GENERAL FUNCTIONS

In [2]:
#Function to show image
def show_img(image):
    plt.imshow(image, interpolation=None)
    plt.grid(None)
    plt.show()

In [3]:
#Image cropping
def crop_image(images_list, nbPix = 100):
    output_images = []
    for image in images_list:
        #Height adjustments
        h = len(image)
        adj = len(image) - nbPix
        h1 = round(adj / 2) #Top
        h2 = h - (adj - h1) #Bottom

        #Width adjustments
        w = len(image[0])
        w_adj = w - nbPix
        w1 = round(w_adj / 2) #Left
        w2 = w - (w_adj - w1) #Right

        img = image[h1:h2,w1:w2]
        output_images.append(img)
        
    return np.array(output_images)

## 2) IMPORT DATA

### 2.1 - Declare file paths

In [4]:
#General file paths
projectDir = os.getcwd() + "/"
parentDir = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/"
dataPath = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/isic-2024-challenge/"

#Metadata file paths
#metaPath = dataPath + "train-metadata.csv"
metaPath = dataPath + "sample-metadata.csv"

#Image file path
#hdf5_file = dataPath + "train-image.hdf5"
hdf5_file = dataPath + "sample-image.hdf5"

In [5]:
#base_path = "C:/Users/admin/Documents/DSTI/DeepLearning/Project/Computer_Vision/isic-2024-challenge"
metadata = pd.read_csv(metaPath, sep=",")
#hdf5_file = os.path.join(base_path, "sampleclaire-image.hdf5")

### 2.2 - Load metadata from csv

In [6]:
#METADATA: color and size features having no NAs
metadata = metadata[["isic_id",
                     "age_approx",
                     "target",
                     "clin_size_long_diam_mm",
                     "tbp_lv_areaMM2",
                     "tbp_lv_area_perim_ratio",
                     "tbp_lv_eccentricity",
                     "tbp_lv_minorAxisMM",
                     "tbp_lv_color_std_mean",
                     "tbp_lv_deltaLBnorm",
                     "tbp_lv_radial_color_std_max",
                     "tbp_lv_location"]]



#Verify that there are no NAs
print("-- X_meta NA counts --")
print(metadata.isna().sum())

#Check number of Unknoxn for tbp_lv_location
loc_unknown=metadata[metadata["tbp_lv_location"]=="Unknown"]
print("Number of unknown for tbp_lv_location", len(loc_unknown))

-- X_meta NA counts --
isic_id                         0
age_approx                     34
target                          0
clin_size_long_diam_mm          0
tbp_lv_areaMM2                  0
tbp_lv_area_perim_ratio         0
tbp_lv_eccentricity             0
tbp_lv_minorAxisMM              0
tbp_lv_color_std_mean           0
tbp_lv_deltaLBnorm              0
tbp_lv_radial_color_std_max     0
tbp_lv_location                 0
dtype: int64
Number of unknown for tbp_lv_location 77


In [7]:
#Activate for debugging of the predict function
#metadata["target_cheat"] = metadata["target"]

### 2.3 - Clean data

In [8]:
metadata=metadata[metadata["tbp_lv_location"]!="Unknown"]

loc_unknown2=metadata[metadata["tbp_lv_location"]=="Unknown"]
print("Number of unknown for tbp_lv_location", len(loc_unknown2))


Number of unknown for tbp_lv_location 0


In [9]:
#Apply One-hot encoding for location
location=pd.get_dummies(metadata["tbp_lv_location"],prefix='category')
location = location.astype(int)
metadata = pd.concat([metadata, location], axis=1)
metadata=metadata.drop("tbp_lv_location",axis=1)
print(metadata.columns)

Index(['isic_id', 'age_approx', 'target', 'clin_size_long_diam_mm',
       'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_eccentricity',
       'tbp_lv_minorAxisMM', 'tbp_lv_color_std_mean', 'tbp_lv_deltaLBnorm',
       'tbp_lv_radial_color_std_max', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third', 'category_Torso Front Bottom Half',
       'category_Torso Front Top Half'],
      dtype='object')


In [10]:
# Calculate the mean of age_approx for each target group
mean_age_malign = metadata.loc[metadata["target"] == 1, "age_approx"].mean()
mean_age_benign = metadata.loc[metadata["target"] == 0, "age_approx"].mean()

# Define a function to fill NA based on the target value
def fill_na_by_target(row):
    if pd.isna(row['age_approx']):
        if row['target'] == 1:
            return mean_age_malign
        elif row['target'] == 0:
            return mean_age_benign
    return row['age_approx']

# Apply the function to the age_approx column
metadata['age_approx'] = metadata.apply(fill_na_by_target, axis=1)

#Verify that there are no NAs
print("-- X_meta NA counts --")
print(metadata.isna().sum())


-- X_meta NA counts --
isic_id                             0
age_approx                          0
target                              0
clin_size_long_diam_mm              0
tbp_lv_areaMM2                      0
tbp_lv_area_perim_ratio             0
tbp_lv_eccentricity                 0
tbp_lv_minorAxisMM                  0
tbp_lv_color_std_mean               0
tbp_lv_deltaLBnorm                  0
tbp_lv_radial_color_std_max         0
category_Head & Neck                0
category_Left Arm                   0
category_Left Arm - Lower           0
category_Left Arm - Upper           0
category_Left Leg                   0
category_Left Leg - Lower           0
category_Left Leg - Upper           0
category_Right Arm                  0
category_Right Arm - Lower          0
category_Right Arm - Upper          0
category_Right Leg                  0
category_Right Leg - Lower          0
category_Right Leg - Upper          0
category_Torso Back Bottom Third    0
category_Torso Back Middle 

In [11]:
'''
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#Normalization
#Select the column
feature=metadata.drop(columns=['isic_id','target'])

#scaler=StandardScaler() for standardization
scaler = MinMaxScaler()
feature_standardized=scaler.fit_transform(feature)
feature_standardized_df = pd.DataFrame(feature_standardized, columns=feature.columns)

metadata=pd.concat([metadata[['isic_id','target']].reset_index(drop=True), feature_standardized_df] , axis=1)
print(len(metadata.columns))
'''

"\n#from sklearn.preprocessing import StandardScaler\nfrom sklearn.preprocessing import MinMaxScaler\n#Normalization\n#Select the column\nfeature=metadata.drop(columns=['isic_id','target'])\n\n#scaler=StandardScaler() for standardization\nscaler = MinMaxScaler()\nfeature_standardized=scaler.fit_transform(feature)\nfeature_standardized_df = pd.DataFrame(feature_standardized, columns=feature.columns)\n\nmetadata=pd.concat([metadata[['isic_id','target']].reset_index(drop=True), feature_standardized_df] , axis=1)\nprint(len(metadata.columns))\n"

In [12]:
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#Normalization
#Select the column
'''
feature=metadata.drop(columns=['isic_id','target', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third','category_Torso Front',
       'category_Torso Front Bottom Half', 'category_Torso Front Top Half'])
'''
#Select the column
feature=metadata.drop(columns=['isic_id','target', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third',
       'category_Torso Front Bottom Half', 'category_Torso Front Top Half'])
#scaler=StandardScaler() for standardization
scaler = MinMaxScaler()
feature_standardized=scaler.fit_transform(feature)
feature_standardized_df = pd.DataFrame(feature_standardized, columns=feature.columns)

'''
metadata=pd.concat([metadata[['isic_id','target', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third', 'category_Torso Front',
       'category_Torso Front Bottom Half', 'category_Torso Front Top Half']].reset_index(drop=True), feature_standardized_df] , axis=1)
'''

metadata=pd.concat([metadata[['isic_id','target', 'category_Head & Neck',
       'category_Left Arm', 'category_Left Arm - Lower',
       'category_Left Arm - Upper', 'category_Left Leg',
       'category_Left Leg - Lower', 'category_Left Leg - Upper',
       'category_Right Arm', 'category_Right Arm - Lower',
       'category_Right Arm - Upper', 'category_Right Leg',
       'category_Right Leg - Lower', 'category_Right Leg - Upper',
       'category_Torso Back Bottom Third', 'category_Torso Back Middle Third',
       'category_Torso Back Top Third',
       'category_Torso Front Bottom Half', 'category_Torso Front Top Half']].reset_index(drop=True), feature_standardized_df] , axis=1)
print(len(metadata.columns))

29


### 2.4a - Train, Validate, Test Split

In [13]:
def list_if_needed(obj):
    if isinstance(obj, pd.Series):
        return obj.tolist()
    return obj

#Function to perform train-validate or train-test-validate split on a list of isic_ids
def ttv_split(isic_ids, test_frac=0.2, validate_frac=0.2, random_state=88, shuffle=True, stratify=None):
    if test_frac < 0 or validate_frac < 0:
        print("ERROR: Test of validate fraction is negative")
        return None
    if test_frac > 1 or validate_frac > 1:
        print("ERROR: Test of validate fraction is above 0")
        return None
    if test_frac + validate_frac >= 1:
        print("ERROR: Test and validate fractions sum to 1 or more.")
        return None

    #Split training from the rest
    test_size = test_frac + validate_frac
    train, temp = train_test_split(isic_ids, test_size = test_size, random_state=random_state, shuffle=shuffle, stratify=stratify)
    #Split test and validate
    if test_frac == 0 or validate_frac == 0:
       # return train.tolist(), temp.tolist()
        return list_if_needed(train), list_if_needed(temp)
    else:
        test_size = test_frac / (test_frac + validate_frac)
        test, validate = train_test_split(temp, test_size = test_size, random_state=random_state, shuffle=shuffle, stratify=stratify)
        #return train.tolist(), test.tolist(), validate.tolist()
        return list_if_needed(train), list_if_needed(test), list_if_needed(validate)

#Generate the splits of the isic_ids
#train_ids, test_ids, val_ids = ttv_split(metadata["isic_id"])

In [14]:
#keep 10% of isic_Id of target=1 without duplication
isic_id_val_target_1 = metadata[metadata['target'] == 1]['isic_id'].tolist()
n=int(0.1*len(isic_id_val_target_1))
isic_ids_keep_train = np.random.choice(isic_id_val_target_1, n , replace=False)


# Split to isolate the test set
isic_ids = metadata.loc[~metadata["isic_id"].isin(isic_ids_keep_train), 'isic_id']
train_val_ids, test_ids= ttv_split(isic_ids, test_frac=0.2, validate_frac=0.0)

In [15]:
print(len(test_ids))

1039


### 2.4b - Data augmentation
- Augment only the malignant data in the training set
- Reformat all lists (train_ids, test_ids, val_ids) to be compatible: list of tuples

In [16]:
#Make list of ids compatible with data augmentations
#Base data takes a value of 0, meaning it should not be modified
train_val_ids_mods = [(id, 0) for id in train_val_ids]
test_ids_mods = [(id, 0) for id in test_ids]
#train_ids_mods = [(id, 0) for id in train_ids]
#val_ids_mods = [(id, 0) for id in val_ids]

In [17]:
#Identify the malignant cases in the training data
#all_pos = metadata[metadata["target"]==1]["isic_id"]
all_pos = metadata.loc[(metadata["target"] == 1) & (~metadata["isic_id"].isin(isic_ids_keep_train)), 'isic_id']
pos_in_train_val = all_pos[all_pos.isin(train_val_ids)]
print("Number of positives in training data:", len(pos_in_train_val))
#pos_in_train = all_pos[all_pos.isin(train_ids)]
#print("Number of positives in training data:", len(pos_in_train))

Number of positives in training data: 209


In [18]:
print(len(metadata))

5223


In [19]:
print(len(train_val_ids))

4154


In [20]:
print(len(train_val_ids_mods))

4154


In [21]:
# Apply augmentations only to training and validation sets before splitting them
#Duplicates of ids will each have a different number, indicating a specific augmentation to be used
nb_of_augments = 50 #apply 500 with the all dataset

rng = np.random.default_rng()
for i in range(nb_of_augments):
    rand_nb = rng.random()
    #Option 1: use random float between 0 and 1
    #train_ids_mods += [(id, rand_nb) for id in pos_in_train_val]
    #Option 2: use integer
    train_val_ids_mods += [(id, i + 1) for id in pos_in_train_val]

#Shuffle the list
np.random.shuffle(train_val_ids_mods)
print(len(train_val_ids_mods))

8334


In [22]:
train_ids_mods, val_ids_mods= ttv_split(train_val_ids_mods, test_frac=0.0, validate_frac=0.33)

In [23]:
print(len(train_ids_mods))

5583


In [24]:
print(len(val_ids_mods))

2751


In [25]:
#apply duplication on reserved validation data (target 1) 

nb_of_augments = 25 #apply 25 with the all dataset

rng = np.random.default_rng()
isic_ids_keep_train_mods=[]
for i in range(nb_of_augments):
    rand_nb = rng.random()
    #Option 1: use random float between 0 and 1
    #train_ids_mods += [(id, rand_nb) for id in pos_in_train_val]
    #Option 2: use integer
    isic_ids_keep_train_mods += [(id, i + 1) for id in isic_ids_keep_train]



#val_size = int(0.5 * len(val_ids_mods))
#random_sample = random.sample(val_ids_mods, val_size)

#Shuffle the list
val_ids_mods=isic_ids_keep_train_mods + val_ids_mods
np.random.shuffle(val_ids_mods)

In [26]:
print(len(val_ids_mods))

3501


### 2.5 - Load images and create hybrid tensorflow dataset

In [27]:
def hair_removal(image, crop_pixels=10):
    height_pixels = len(image)  # Image rows
    width_pixels = len(image[0])  # Image columns

    # Image cropping
    height = [crop_pixels, height_pixels - crop_pixels]
    width = [crop_pixels, width_pixels - crop_pixels]
    img = image[height[0]:height[1], width[0]:width[1]]

    # Gray scale
    grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Black hat filter
    kernel = cv2.getStructuringElement(1, (9, 9))
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    # Gaussian filter
    bhg = cv2.GaussianBlur(blackhat, (3, 3), cv2.BORDER_DEFAULT)
    # Binary thresholding (MASK)
    ret, mask = cv2.threshold(bhg, 10, 255, cv2.THRESH_BINARY)
    # Replace pixels of the mask
    dst = cv2.inpaint(img, mask, 6, cv2.INPAINT_TELEA)

    return dst

#def resize_image(image, target_size=(100, 100)):
#    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
#    return resized_image

In [28]:
# Define the augmentation function
def augment_image(image):
    """
    Apply a series of augmentations to create diverse variations of the input image.
    Includes random flips, rotations, brightness adjustments, and other transformations.
    """
    # Apply various augmentations
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    return image

In [29]:
def compute_class_weights(metadata, img_names):
    # Initialize counters for target=0 and target=1
    target_0_count = 0
    target_1_count = 0

    # Loop through each tuple in img_names (img_name, transformation)
    for img_name, mod in img_names:
        # Filter metadata to find the corresponding isic_id
        metadata_filtered = metadata[metadata["isic_id"] == img_name]

        if not metadata_filtered.empty:
            # Retrieve the target value for the corresponding isic_id
            target = metadata_filtered["target"].values[0]

            # Increment the counter based on the target value
            if target == 0:
                target_0_count += 1
            elif target == 1:
                target_1_count += 1

    # Calculate total number of images
    total = target_0_count + target_1_count

    # Calculate class weights based on the counts, avoid division by zero
    if target_0_count > 0:
        weight_for_0 = total / (2 * target_0_count)
    else:
        weight_for_0 = 1

    if target_1_count > 0:
        weight_for_1 = total / (2 * target_1_count)
    else:
        weight_for_1 = 1

    return weight_for_0, weight_for_1

In [30]:
# GENERATOR FOR HDF5 AND METADATA
"""
file = filepath to the hdf5 file containing the image data
metadata = the full metadata dataframe (col1 = "isic_id", col2 = "target")
img_names = list of tuples, each containing the isic_id followed by a number, signifying:
            * 0 when original data is to be used
            * random number - data augmentation is to be applied
imgSize = images are to be adjusted to this size (square) in pixels
"""
class hdf5_generator:
    def __init__(self, file, metadata, img_names, imgSize,is_training=False):
        self.file = file
        self.metadata = metadata
        self.img_names = img_names
        self.imgSize = imgSize
        self.is_training=is_training

    def __call__(self):
        with h5py.File(self.file, 'r') as h5file:
            for img_name_tuple in self.img_names:
                img_name, mod = img_name_tuple
                try:
                    # Load image data from HDF5
                    img = np.array(Image.open(io.BytesIO(h5file[img_name][()])))
                    
                    # Clean image
                    img = hair_removal(img)

                    
                    #if mod != 0:
                    if self.is_training==True:
                        # Data Augmentation 
                        img= augment_image(img)
                    else:
                        img=img
                        
                    # Resize the image
                    img = tf.image.resize(img, [self.imgSize, self.imgSize])

                    # Standardize and return as TensorFlow constant
                    img = tf.constant(img / 255, dtype=tf.float32)  # Standardize here

                    #Retrieve corresponding metadata
                    meta = self.metadata[self.metadata["isic_id"] == img_name].iloc[:,2:]

                    #Retrieve corresponding target
                    target = self.metadata[self.metadata["isic_id"] == img_name]["target"]
                    target = np.reshape(target, (1, 1))
                    
                    yield (img, meta), target
                    
                except Exception as e:
                    print(f"Error loading image {img_name}: {e}")
                    # log the error to a file for later analysis
                    with open('image_errors.log', 'a') as f:
                        f.write(f"Error loading image {img_name}: {e}\n")
                    continue

#Generate the dataset with batch size and prefetching
def make_dataset(hdf5_file, metadata, img_names, imgSize=100, batch_size=32, shuffle=True, is_training=False):
    if is_training:
        #Get number of picture for each target
        weight_for_0, weight_for_1 = compute_class_weights(metadata, img_names)
        #print(f"Class weight for target=0: {weight_for_0}")
        #print(f"Class weight for target=1: {weight_for_1}")
    else:
        weight_for_0, weight_for_1 = None, None
        
    # Get the number of metadata features (isic_id and target are present, so subtract)
    num_features = metadata.shape[-1] - 2
    
    # Generate image dataset
    element_spec = ((tf.TensorSpec(shape=(imgSize, imgSize, 3), dtype=tf.float32),
                 tf.TensorSpec(shape=(1, num_features), dtype=tf.float32)),
                tf.TensorSpec(shape=(1, 1), dtype=tf.int32))
    
    img_dataset = tf.data.Dataset.from_generator(
        hdf5_generator(hdf5_file, metadata, img_names, imgSize, is_training),
        output_signature=element_spec
    )

    # Add shuffling, batching, and prefetching
    dataset = img_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    return dataset, (weight_for_0, weight_for_1)

## 3) CNN MODEL

### 3.1 - Model class

In [31]:
#Simple CNN model using only images and target
class CNN_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'leaky_relu', img_size = 100, img_channels=3):
        #Run the constructor of the parent class
        super().__init__()

        #Weight and bias initializers
        kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
        bias_initializer = tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)
        
        #Image size declaration
        self.img_size = img_size
        self.img_channels = img_channels

        #Layers
        self.conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu', padding='same', input_shape=(img_size, img_size, img_channels),
                                            kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)

    def call(self, inputs):
        x_image, x_meta = inputs

        # Convolutions
        x1 = self.conv1(x_image)
        x1 = self.pool1(x1)

        # Flattening of images for input layer
        x1 = self.flatten(x1)

        # Hidden layers of neural network
        x1 = self.dense1(x1)

        # Output layer of neural network
        output = self.dense2(x1)

        return output

#Metadata Neural Network
class Meta_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'tanh'):
        #Run the constructor of the parent class
        super().__init__()

        #Weight and bias initializers
        kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
        bias_initializer = tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

        #Layers
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.dense2 = tf.keras.layers.Dense(neurons, activation = activ, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.dropout = tf.keras.layers.Dropout(0.25)

    def call(self, inputs, training=False):
        x_image, x_meta = inputs
        x_all = tf.reshape(x_meta, (tf.shape(x_meta)[0], x_meta.shape[-1]))
        # Neural Network
        x_all = self.dense1(x_all)
        x_all = self.dense2(x_all)
        if training:
            x_all = self.dropout(x_all, training=training)
        output = self.dense3(x_all)
        return output

#Hybrid CNN model taking metadata
class Hybrid_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'leaky_relu', img_size = 100, img_channels = 3):
        #Run the constructor of the parent class
        super().__init__()

        #Weight and bias initializers
        kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
        bias_initializer = tf.keras.initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None)

        #Image size declaration
        self.img_size = img_size
        self.img_channels = img_channels

        #Layers
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=5, strides=(1, 1), activation='relu', padding='same', input_shape=(img_size, img_size, img_channels),
                                            kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.conv2 = tf.keras.layers.Conv2D(64, 5, activation='relu', kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.dropout1 = tf.keras.layers.Dropout(0.10)
        self.dense2 = tf.keras.layers.Dense(neurons, activation = activ, kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.dropout2 = tf.keras.layers.Dropout(0.10)
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=kernel_initializer, bias_initializer=bias_initializer)
        self.concatenate = keras.layers.Concatenate(axis=1)
        
    def call(self, inputs, training=False):
        x_image, x_meta = inputs
        # Convolutions
        x = self.conv1(x_image)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        # Flattening of images and concatenation with other data
        x = self.flatten(x)
        # Reshape metadata to match dimensions
        x_meta = tf.reshape(x_meta, (tf.shape(x_meta)[0], x_meta.shape[-1]))
        x_all = self.concatenate([x, x_meta])
        # Neural Network
        x_all = self.dense1(x_all)
        if training:
            x_all = self.dropout1(x_all, training=training)
        x_all = self.dense2(x_all)
        if training:
            x_all = self.dropout2(x_all, training=training)
        output = self.dense3(x_all)
        return output

### 3.2 - Model compiling

In [32]:
#Set seed
tf.random.set_seed(71)

#Initialize model
#model = CNN_model(neurons=8, activ='tanh')
model = Hybrid_model(neurons=36, activ='leaky_relu')
#model = Meta_model(neurons=18, activ='tanh')

#Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False,
                                          label_smoothing=0.0,
                                          axis=-1,
                                          reduction='sum_over_batch_size',
                                          name='binary_crossentropy')

#Compile the model with loss, optimizer, and metrics
model.compile(loss = loss,
              optimizer = optimizer,
              metrics = [
                  tf.keras.metrics.BinaryAccuracy(),
                  tf.keras.metrics.FalseNegatives(),
                  tf.keras.metrics.FalsePositives(),
                  tf.keras.metrics.TrueNegatives(),
                  tf.keras.metrics.TruePositives()
                  ]
)

2024-09-23 20:02:52.290631: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [33]:
"""
# Take 1 batch from the dataset and check its content
for batch in train_dataset.take(1):
    (img_batch, meta_batch), target_batch = batch
    
    # Print the shapes of the individual components
    print(f"Image batch shape: {img_batch.shape}")
    print(f"Metadata batch shape: {meta_batch.shape}")
    print(f"Target batch shape: {target_batch.shape}")

# To count the total number of batches
batch_count = 0
for _ in train_dataset:
    batch_count += 1

print(f"Total number of batches in the dataset: {batch_count}")
"""

'\n# Take 1 batch from the dataset and check its content\nfor batch in train_dataset.take(1):\n    (img_batch, meta_batch), target_batch = batch\n    \n    # Print the shapes of the individual components\n    print(f"Image batch shape: {img_batch.shape}")\n    print(f"Metadata batch shape: {meta_batch.shape}")\n    print(f"Target batch shape: {target_batch.shape}")\n\n# To count the total number of batches\nbatch_count = 0\nfor _ in train_dataset:\n    batch_count += 1\n\nprint(f"Total number of batches in the dataset: {batch_count}")\n'

### 3.3 - Model fit

In [34]:
#Clear the memory leak in Keras
class CustomCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()
    #print(f"Epoch {epoch+1} finished. Validation loss: {logs['val_loss']}")

In [35]:
#Set batch sizes
train_batch_size = 32
val_batch_size = 1
test_batch_size = 1

#Determine the number of batches
nb_training_batches = int(len(train_ids_mods)//train_batch_size)
nb_validate_batches = int(len(val_ids_mods)//val_batch_size)
nb_test_batches = int(len(val_ids_mods)//test_batch_size)

#Print results
print("Total training batches in dataset:", nb_training_batches)
print("Total validate batches in dataset:", nb_validate_batches)
print("Total test batches in dataset:", nb_test_batches)

Total training batches in dataset: 174
Total validate batches in dataset: 3501
Total test batches in dataset: 3501


In [36]:
#Create datasets and get weights
train_dataset, class_weights = make_dataset(hdf5_file, metadata, train_ids_mods, batch_size=train_batch_size, is_training=True)
weight_for_0, weight_for_1 = class_weights
validate_dataset, _ = make_dataset(hdf5_file, metadata, val_ids_mods, batch_size = val_batch_size, is_training=False)
test_dataset, _ = make_dataset(hdf5_file, metadata, test_ids_mods, batch_size=test_batch_size)

: 

In [37]:
#Run the model through epochs
for epoch in range(25):
    #Make datasets
    #np.random.seed(87 + i)
    np.random.shuffle(train_ids_mods)
    print("First training ID:", train_ids_mods[0][0])
    train_dataset, _ = make_dataset(hdf5_file, metadata, train_ids_mods, batch_size=train_batch_size, is_training=True)
    validate_dataset, _ = make_dataset(hdf5_file, metadata, val_ids_mods, batch_size = val_batch_size, is_training=False)

    mod = model.fit(train_dataset, epochs=1, steps_per_epoch = nb_training_batches, validation_data = validate_dataset, validation_steps = nb_validate_batches, callbacks = [CustomCallback()],
                    class_weight={0: weight_for_0, 1: weight_for_1})

First training ID: ISIC_5506776


/home/awieber/miniconda3/envs/ISIC24_skin_cancer/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
/home/awieber/miniconda3/envs/ISIC24_skin_cancer/lib/python3.11/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


174/174 [==============================] - 342s 2s/step - loss: 0.7019 - binary_accuracy: 0.6252 - false_negatives: 1239.0000 - false_positives: 848.0000 - true_negatives: 1777.0000 - true_positives: 1704.0000 - val_loss: 0.5623 - val_binary_accuracy: 0.7107 - val_false_negatives: 704.0000 - val_false_positives: 309.0000 - val_true_negatives: 1007.0000 - val_true_positives: 1481.0000
First training ID: ISIC_8215425
174/174 [==============================] - 345s 2s/step - loss: 0.5446 - binary_accuracy: 0.7292 - false_negatives: 1029.0000 - false_positives: 479.0000 - true_negatives: 2142.0000 - true_positives: 1918.0000 - val_loss: 0.5111 - val_binary_accuracy: 0.7286 - val_false_negatives: 746.0000 - val_false_positives: 204.0000 - val_true_negatives: 1112.0000 - val_true_positives: 1439.0000
First training ID: ISIC_9709310
174/174 [==============================] - 394s 2s/step - loss: 0.4778 - binary_accuracy: 0.7870 - false_negatives: 689.0000 - false_positives: 497.0000 - true_ne

In [45]:
#mod = model.fit(train_dataset, epochs=25, validation_data = validate_dataset,class_weight={0: weight_for_0, 1: weight_for_1})

Epoch 1/25
     46/Unknown 49s 926ms/step - binary_accuracy: 0.5159 - false_negatives: 112.7609 - false_positives: 228.1739 - loss: 1.0996 - true_negatives: 323.1304 - true_positives: 87.7826

C:\Users\admin\anaconda3\envs\env_DL_python3119\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


46/46 ━━━━━━━━━━━━━━━━━━━━ 59s 1s/step - binary_accuracy: 0.5161 - false_negatives: 114.9787 - false_positives: 233.4894 - loss: 1.0937 - true_negatives: 328.8723 - true_positives: 89.6809 - val_binary_accuracy: 0.3506 - val_false_negatives: 0.0000e+00 - val_false_positives: 502.0000 - val_loss: 0.6986 - val_true_negatives: 3.0000 - val_true_positives: 268.0000
Epoch 2/25
46/46 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - binary_accuracy: 0.3494 - false_negatives: 18.3830 - false_positives: 449.4043 - loss: 0.6916 - true_negatives: 112.9574 - true_positives: 186.2766 - val_binary_accuracy: 0.7827 - val_false_negatives: 139.0000 - val_false_positives: 29.0000 - val_loss: 0.6565 - val_true_negatives: 476.0000 - val_true_positives: 129.0000
Epoch 3/25
46/46 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - binary_accuracy: 0.7507 - false_negatives: 88.0000 - false_positives: 91.4894 - loss: 1.7777 - true_negatives: 470.8723 - true_positives: 116.6596 - val_binary_accuracy: 0.7930 - val_false_negatives: 86.0000 - 

In [46]:
model.weights

[<KerasVariable shape=(5, 5, 3, 32), dtype=float32, path=hybrid_model/conv2d/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=hybrid_model/conv2d/bias>,
 <KerasVariable shape=(5, 5, 32, 64), dtype=float32, path=hybrid_model/conv2d_1/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=hybrid_model/conv2d_1/bias>,
 <KerasVariable shape=(33883, 36), dtype=float32, path=hybrid_model/dense/kernel>,
 <KerasVariable shape=(36,), dtype=float32, path=hybrid_model/dense/bias>,
 <KerasVariable shape=(36, 36), dtype=float32, path=hybrid_model/dense_1/kernel>,
 <KerasVariable shape=(36,), dtype=float32, path=hybrid_model/dense_1/bias>,
 <KerasVariable shape=(36, 1), dtype=float32, path=hybrid_model/dense_2/kernel>,
 <KerasVariable shape=(1,), dtype=float32, path=hybrid_model/dense_2/bias>]

In [47]:
mod.history

{'binary_accuracy': [0.5255972743034363,
  0.49897611141204834,
  0.7665529251098633,
  0.8184300065040588,
  0.8470989465713501,
  0.8600682616233826,
  0.900341272354126,
  0.9044368863105774,
  0.920819103717804,
  0.9317406415939331,
  0.9378839731216431,
  0.9447098970413208,
  0.9481228590011597,
  0.9467576742172241,
  0.9535835981369019,
  0.9542661905288696,
  0.950170636177063,
  0.9515358209609985,
  0.9569965600967407,
  0.958361804485321,
  0.9549487829208374,
  0.9610921740531921,
  0.9617747664451599,
  0.9617747664451599,
  0.9638225436210632],
 'false_negatives': [217.0,
  62.0,
  141.0,
  54.0,
  33.0,
  33.0,
  15.0,
  18.0,
  12.0,
  7.0,
  12.0,
  9.0,
  8.0,
  6.0,
  4.0,
  6.0,
  6.0,
  7.0,
  5.0,
  5.0,
  4.0,
  3.0,
  5.0,
  3.0,
  3.0],
 'false_positives': [478.0,
  672.0,
  201.0,
  212.0,
  191.0,
  172.0,
  131.0,
  122.0,
  104.0,
  93.0,
  79.0,
  72.0,
  68.0,
  72.0,
  64.0,
  61.0,
  67.0,
  64.0,
  58.0,
  56.0,
  62.0,
  54.0,
  51.0,
  53.0,
  50.0

**BATCHES**

In [ ]:
"""
# Iterate through all batches in the dataset and print their shapes
for i, batch in enumerate(train_dataset):
    (img_batch, meta_batch), target_batch = batch
    
    # Print the shapes of the current batch
    print(f"Batch {i+1}:")
    print("  Image Batch Shape:", img_batch.shape)
    print("  Metadata Batch Shape:", meta_batch.shape)
    print("  Target Batch Shape:", target_batch.shape)
"""

### 3.4 - Predict Test Data

In [ ]:
#Retrieve test predictions and real test values
predictions = model.predict(test_dataset, steps = nb_test_batches)
y_pred = [round(i) for i  in predictions.flatten()]
y_test = np.concatenate([y for x, y in test_dataset], axis=0).flatten()

In [ ]:
#Calculate the loss
loss = sum(abs(y_test - y_pred))/len(y_pred)
print("Shape of prediction data:", predictions.shape)
print("Loss on test data:", loss)